In [ ]:
import math
import copy
import random
import numpy as np
import pickle
import heapq
import matplotlib.pyplot as plt

### Helper functions

In [ ]:
def gridToImage(grid, color_dict):
    grid = copy.deepcopy(grid)
    for row in grid:
        for i in range(len(row)):
            row[i] = color_dict.get(row[i], [0, row[i], 0])
    plt.imshow(grid)
    plt.show()

def showPath(start, end, grid, path):
    grid = copy.deepcopy(grid)
    for n in path:
        grid[n[1]][n[0]] = 7

    grid[start[1]][start[0]] = 'S'
    grid[end[1]][end[0]] = 'E'

    color_dict = {
        0 : [255, 255, 255],
        1 : [0, 0, 0],
        7 : [0, 255, 0],
        'S' : [0, 0, 255],
        'E' : [255, 0, 0]
    }

    gridToImage(grid, color_dict)

def showValues(start, end, grid):
    grid = copy.deepcopy(grid)
    max_value = max([max(row) for row in grid])
    for row in grid:
        for i in range(len(row)):
            if isinstance(row[i], int) and row[i] == 1:
                row[i] = 'O'
            elif isinstance(row[i], float):
                row[i] /= max_value

    grid[start[1]][start[0]] = 'S'
    grid[end[1]][end[0]] = 'E'

    color_dict = {
        0 : [255, 255, 255],
        'O' : [0, 0, 0],
        7 : [0, 255, 0],
        'S' : [0, 0, 255],
        'E' : [255, 0, 0]
    }

    gridToImage(grid, color_dict)

#### Create cost grid

In [ ]:
def get_distance(start, end):
    return math.hypot((start[0] - end[0]), (start[1] - end[1]))

In [ ]:
def create_cost_grid(grid, check_range, k):
    new_grid = copy.deepcopy(grid)

    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == 1:
                if c == 0 or grid[r][c - 1] == 1:
                    left_check = 0
                else:
                    left_check = -check_range

                if c == len(grid[0]) - 1 or grid[r][c + 1] == 1:
                    right_check = 0
                else:
                    right_check = check_range

                if r == len(grid) - 1 or grid[r + 1][c] == 1:
                    up_check = 0
                else:
                    up_check = check_range

                if r == 0 or grid[r - 1][c] == 1:
                    down_check = 0
                else:
                    down_check = -check_range

                for dx in range(left_check, right_check + 1):
                    for dy in range(down_check, up_check + 1):
                        x, y = c + dx, r + dy
                        if 0 <= x < len(grid[0]) and 0 <= y < len(grid) and grid[y][x] != 1:
                            dist = get_distance((0, 0), (dx, dy))
                            cost = k * (1 / dist - 1 / check_range)
                            new_grid[y][x] = max(new_grid[y][x], cost)

    return new_grid

## Pathfinding Algorithms

### Dijkstra

In [ ]:
def dijkstra(start, end, graph, is_diagonal=True):
    
    if graph[start[1]][start[0]] == 1 or graph[end[1]][end[0]] == 1:
        return []

    open_list = []
    closed_list = set()
    open_set = {}

    # f_score, g_score, pos
    start_node = (0, 0, start)
    index = 0
    open_list.append((0, index, start_node))
    heapq.heapify(open_list)

    parent = {}

    if is_diagonal:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, -1), (-1, 1)]
    else:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    while open_list:
        start_f_score, start_g_score, start_pos = heapq.heappop(open_list)[-1]
        closed_list.add(start_pos)

        if start_pos[0] == end[0] and start_pos[1] == end[1]:
            pos = start_pos
            path = []
            while pos in parent:
                path.append(pos)
                pos = parent[pos]
            path.append(start)

            path = path[::-1]
            return path

        for dir in dirs:
            x = dir[0]
            y = dir[1]

            pos = (start_pos[0] + x, start_pos[1] + y)

            if pos in closed_list:
                continue

            if pos[0] < 0:
                continue
            if pos[0] >= len(graph[0]):
                continue
            if pos[1] < 0:
                continue
            if pos[1] >= len(graph):
                continue

            if graph[pos[1]][pos[0]] == 1:
                continue

            # g_score = path to n
            g_score = start_g_score + get_distance(start_pos, pos)

            h_score = get_distance(end, pos)
            f_score = g_score + h_score

            if pos in open_set:
                if open_set[pos] <= g_score:
                    continue

            open_set[pos] = g_score
            parent[pos] = start_pos
            index += 1
            heapq.heappush(open_list, (f_score, index, (f_score, g_score, pos)))

    return []

### A* Search

In [ ]:
def astar(start, end, graph, is_diagonal=True):
    
    if graph[start[1]][start[0]] == 1 or graph[end[1]][end[0]] == 1:
        return []

    open_list = []
    closed_list = set()
    open_set = {}

    # f_score, g_score, pos
    start_node = (0, 0, start)
    index = 0
    open_list.append((0, index, start_node))
    heapq.heapify(open_list)

    parent = {}

    if is_diagonal:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, -1), (-1, 1)]
    else:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    while open_list:
        start_f_score, start_g_score, start_pos = heapq.heappop(open_list)[-1]
        closed_list.add(start_pos)

        if start_pos[0] == end[0] and start_pos[1] == end[1]:
            pos = start_pos
            path = []
            while pos in parent:
                path.append(pos)
                pos = parent[pos]
            path.append(start)

            path = path[::-1]
            return path

        for dir in dirs:
            x = dir[0]
            y = dir[1]

            pos = (start_pos[0] + x, start_pos[1] + y)

            if pos in closed_list:
                continue

            if pos[0] < 0:
                continue
            if pos[0] >= len(graph[0]):
                continue
            if pos[1] < 0:
                continue
            if pos[1] >= len(graph):
                continue

            if graph[pos[1]][pos[0]] == 1:
                continue

            # g_score = path to n
            g_score = start_g_score + get_distance(start_pos, pos)

            h_score = get_distance(end, pos)
            f_score = g_score + h_score


            if pos in open_set:
                if open_set[pos] <= g_score:
                    continue

            open_set[pos] = g_score
            parent[pos] = start_pos
            index += 1
            heapq.heappush(open_list, (f_score, index, (f_score, g_score, pos)))

    return []

### Dijkstra w/ APF

In [ ]:
def dijkstra_apf(start, end, graph, cost_grid, is_diagonal=True):
    
    if graph[start[1]][start[0]] == 1 or graph[end[1]][end[0]] == 1:
        return []

    open_list = []
    closed_list = set()
    open_set = {}

    # f_score, g_score, pos
    start_node = (0, cost_grid[start[1]][start[0]], start)
    index = 0
    open_list.append((0, index, start_node))
    heapq.heapify(open_list)

    parent = {}

    if is_diagonal:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, -1), (-1, 1)]
    else:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    while open_list:
        start_f_score, start_g_score, start_pos = heapq.heappop(open_list)[-1]
        closed_list.add(start_pos)

        if start_pos[0] == end[0] and start_pos[1] == end[1]:
            pos = start_pos
            path = []
            while pos in parent:
                path.append(pos)
                pos = parent[pos]
            path.append(start)

            path = path[::-1]
            return path

        for dir in dirs:
            x = dir[0]
            y = dir[1]

            pos = (start_pos[0] + x, start_pos[1] + y)

            if pos in closed_list:
                continue

            if pos[0] < 0:
                continue
            if pos[0] >= len(graph[0]):
                continue
            if pos[1] < 0:
                continue
            if pos[1] >= len(graph):
                continue

            if graph[pos[1]][pos[0]] == 1:
                continue

            # g_score = path to n + repulsive force
            g_score = start_g_score + get_distance(start_pos, pos)
            g_score += cost_grid[pos[1]][pos[0]]

            f_score = g_score

            if pos in open_set:
                if open_set[pos] <= g_score:
                    continue

            open_set[pos] = g_score
            parent[pos] = start_pos
            index += 1
            heapq.heappush(open_list, (f_score, index, (f_score, g_score, pos)))

    return []

### A* Search w/ APF

In [ ]:
def astar_apf(start, end, graph, cost_grid, is_diagonal=True):
    
    if graph[start[1]][start[0]] == 1 or graph[end[1]][end[0]] == 1:
        return []

    open_list = []
    closed_list = set()
    open_set = {}

    # f_score, g_score, pos
    start_node = (0, cost_grid[start[1]][start[0]], start)
    index = 0
    open_list.append((0, index, start_node))
    heapq.heapify(open_list)

    parent = {}

    if is_diagonal:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, -1), (-1, 1)]
    else:
        dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    while open_list:
        start_f_score, start_g_score, start_pos = heapq.heappop(open_list)[-1]
        closed_list.add(start_pos)

        if start_pos[0] == end[0] and start_pos[1] == end[1]:
            pos = start_pos
            path = []
            while pos in parent:
                path.append(pos)
                pos = parent[pos]
            path.append(start)

            path = path[::-1]
            return path

        for dir in dirs:
            x = dir[0]
            y = dir[1]

            pos = (start_pos[0] + x, start_pos[1] + y)

            if pos in closed_list:
                continue

            if pos[0] < 0:
                continue
            if pos[0] >= len(graph[0]):
                continue
            if pos[1] < 0:
                continue
            if pos[1] >= len(graph):
                continue

            if graph[pos[1]][pos[0]] == 1:
                continue

            # g_score = path to n + repulsive force
            g_score = start_g_score + get_distance(start_pos, pos)
            g_score += cost_grid[pos[1]][pos[0]]

            h_score = get_distance(end, pos)
            f_score = g_score + h_score


            if pos in open_set:
                if open_set[pos] <= g_score:
                    continue

            open_set[pos] = g_score
            parent[pos] = start_pos
            index += 1
            heapq.heappush(open_list, (f_score, index, (f_score, g_score, pos)))

    return []

## Metric functions

In [ ]:
def calculate_length(path):
    length = 0
    for i in range(1, len(path)):
        length += get_distance(path[i-1], path[i])
    return length

In [ ]:
def binary_array_to_points(binary_array):
    points = []
    for y in range(len(binary_array)):
        for x in range(len(binary_array[y])):
            if binary_array[y][x] != 0:
                points.append((x, y))
    return points

In [ ]:
def calculate_min_distance(path, grid):
    obstacles = binary_array_to_points(grid)
    min_dist_to_path = float('inf')
    for point in path:
        min_dist_to_point = float('inf')
        for obstacle in obstacles:
            min_dist_to_point = min(min_dist_to_point, get_distance(point, obstacle))
        min_dist_to_path = min(min_dist_to_path, min_dist_to_point)
    return min_dist_to_path

In [ ]:
def calculate_avg_min_distance(path, grid):
    obstacles = binary_array_to_points(grid)
    min_dists = []
    for point in path:
        min_dist_to_point = float('inf')
        for obstacle in obstacles:
            min_dist_to_point = min(min_dist_to_point, get_distance(point, obstacle))
        min_dists.append(min_dist_to_point)
    return np.average(min_dists)

## Experiment

In [ ]:
def print_stats(alg_name, path, data):
    print(f'Algorithm: {alg_name}')
    print(f'Length: {calculate_length(path)}')
    print(f'Min dist to path: {calculate_min_distance(path, data)}')
    print(f'Avg min dist to path: {calculate_avg_min_distance(path, data)}')
    print('-' * 7)

def run_all_paths(start, end, graph, is_diagonal=True):
    cost_grid = create_cost_grid(graph, 5, 5)

    path = dijkstra(start, end, graph, is_diagonal)
    print_stats("Dijkstra", path, graph)

    path = astar(start, end, graph, is_diagonal)
    print_stats("A* Search", path, graph)

    path = dijkstra_apf(start, end, graph, cost_grid, is_diagonal)
    print_stats("Dijkstra w/ APF", path, graph)

    path = astar_apf(start, end, graph, cost_grid, is_diagonal)
    print_stats("A* Search w/ APF", path, graph)


In [ ]:
with open('occupancy_map.pickle', "rb") as f:
    original_data = pickle.load(f)

plt.imshow(original_data, cmap='Greys')

In [ ]:
data = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
data = np.array(data, dtype=float)

### Work

In [ ]:
C = 5
data_cost = create_cost_grid(data, C, C*2)
path = astar_apf((0, 10), (19, 10), data, cost_grid=data_cost)

grid = copy.deepcopy(data_cost)
for x, y in path:
    grid[y][x] = -1

plt.imshow(grid, cmap='Blues')

In [ ]:
data = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
data = np.array(data, dtype=float)

In [ ]:
start = (0, 10)
end = (19, 10)

run_all_paths(start, end, data)

C = 5
data_cost = create_cost_grid(data, C, C)
path = astar_apf((0, 10), (19, 10), data, data_cost)

grid = copy.deepcopy(data)
for x, y in path:
    grid[y][x] = -1

plt.imshow(grid, cmap='Blues')

In [ ]:
C = 5
data_cost = create_cost_grid(data, C, C)
path = dijkstra_apf((0, 10), (19, 10), data, data_cost)

grid = copy.deepcopy(data)
for x, y in path:
    grid[y][x] = -1

plt.imshow(grid, cmap='Blues')

In [ ]:
data = original_data.copy()

C = np.random.randint(0, 50)
C = 10
data_cost = create_cost_grid(data, C, C*2)

x, y = np.random.randint(0, 200), np.random.randint(0, 200)
while data[y][x] == 1:
    x, y = np.random.randint(0, 200), np.random.randint(0, 200)
start = (x, y)
x, y = np.random.randint(0, 200), np.random.randint(0, 200)
while data[y][x] == 1:
    x, y = np.random.randint(0, 200), np.random.randint(0, 200)
end = (x, y)

start = (0, 0)
end = (199, 199)

path = astar(start, end, data, data_cost, False, True)
print(f'Length: {calculate_length(path, data)}')
print(f'Min dist to path: {calculate_min_distance(path, data)}')
print(f'Avg min dist to path: {calculate_avg_min_distance(path, data)}')

grid = copy.deepcopy(data_cost)
for x, y in path:
    grid[y][x] = max([max(row) for row in grid])
plt.imshow(grid, cmap='Greys')

In [ ]:
np.random.randint(0, 200)

In [ ]:
data = original_data.copy()

C = 10
data_cost = create_cost_grid(data, C, C)
path = astar((0, 0), (199, 199), data, cost_grid=data_cost)
print(calculate_cost(path, data_cost))

#path = fix_path(path, data, [10, 5, 2.5, 1], 10)

grid = copy.deepcopy(data_cost)
for x, y in path:
    grid[y][x] = max([max(row) for row in grid])
plt.imshow(grid, cmap='Greys')

In [ ]:
data = original_data.copy()

C = 10
data_cost = create_cost_grid(data, C, C)
path = astar((199, 199), (0, 0), data, cost_grid=data_cost)
print(calculate_cost(path, data_cost))

#path = fix_path(path, data, [10, 5, 2.5, 1], 10)

grid = copy.deepcopy(data_cost)
for x, y in path:
    grid[y][x] = max([max(row) for row in grid])
plt.imshow(grid, cmap='Greys')

In [ ]:
data = original_data.copy()

data_cost = create_cost_grid(data, 10)
path = astar((0, 0), (199, 199), data, cost_grid=data_cost)


grid = copy.deepcopy(data)
for x, y in path:
    grid[y][x] = -1
plt.imshow(grid, cmap='Greys')